Optimize and plot the gains/VSWR of a logperiodic antenna (6 brass elements, 75 Ohm transmission lines) for both the 2.4GHz as the 5.8GHz ISM bands.
    Inspired by an excercise for a course, hence the weird constraints.


In [1]:
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt
import matplotlib as mpl

from PyNEC import *
from antenna_util import *

from context_clean import *

import math

brass_conductivity = 15600000 # mhos
copper_conductivity = 1.45e7 # Copper
ground_conductivity = 0.002
ground_dielectric = 10

tl_impedance = 75

In [2]:
def n_seg(freq, length):
  wavelength = 3e8/(1e6*freq)
  return (2 * (int(math.ceil(77*length/wavelength))/2)) + 1


def sc_quad_helix(height, diameter, wire_diameter = 0.02):
    
    nec = context_clean(nec_context())
    nec.set_extended_thin_wire_kernel(True)
    
    geo = geometry_clean(nec.get_geometry())

    wire_r = wire_diameter/2;
    helix_r = diameter/2;
    
    
    #print "Wire Diameter %s" % (wire_r * 2)
    
    helix_turns = 0.5
    
    # helix loop 
    helix_twist_height = height / helix_turns
    geo.helix(tag_id=1, nr_segments=50, spacing=helix_twist_height, lenght=height, start_radius=np.array([helix_r, 0]), end_radius=np.array([helix_r, 0]), wire_radius=wire_r)
    geo.move(rotate_z=90, copies=3, tag_inc=1)
    geo.wire(tag_id=10, nr_segments=2, src=np.array([0, 0, height]), dst=np.array([helix_r, 0, height]), radius=wire_r)
    geo.wire(tag_id=11, nr_segments=2, src=np.array([0, 0, height]), dst=np.array([0, helix_r, height]), radius=wire_r)
    geo.wire(tag_id=12, nr_segments=2, src=np.array([0, 0, height]), dst=np.array([-helix_r, 0, height]), radius=wire_r)
    geo.wire(tag_id=13, nr_segments=2, src=np.array([0, 0, height]), dst=np.array([0, -helix_r, height]), radius=wire_r)
    
    # Everything is copper
    nec.set_wire_conductivity(copper_conductivity)
    # finish structure definition
    nec.geometry_complete(ground_plane=False)

    # Voltage excitation at legs of the antenna
    nec.voltage_excitation(wire_tag=1, segment_nr=1, voltage=1.0)
    nec.voltage_excitation(wire_tag=2, segment_nr=1, voltage=0.0+1.0j)
    nec.voltage_excitation(wire_tag=3, segment_nr=1, voltage=-1.0)
    nec.voltage_excitation(wire_tag=4, segment_nr=1, voltage=0.0-1.0j)
    #nec.set_frequencies_linear(start_frequency=140, stop_frequency=150, count=100)
    #nec.radiation_pattern(thetas=Range(90, 90, count=1), phis=Range(180,180,count=1))

    return nec

In [3]:
start = 100
stop  = 150
count = stop - start

In [4]:
def get_gain_swr_range(height, diameter, start=start, stop=stop, step=1):
    gains_db = []
    frequencies = []
    vswrs = []
    for freq in range(start, stop + 1, step):
        nec = sc_quad_helix(height, diameter)
        nec.set_frequency(freq) # TODO: ensure that we don't need to re-generate this!
        nec.radiation_pattern(thetas=Range(90, 90, count=1), phis=Range(180,180,count=1))

        rp = nec.context.get_radiation_pattern(0)
        ipt = nec.get_input_parameters(0)
        z = ipt.get_impedance()[0]

        # Gains are in decibels
        gains_db.append(rp.get_gain()[0])
        vswrs.append(vswr(z, system_impedance))
        frequencies.append(ipt.get_frequency())

    return frequencies, gains_db, vswrs

def get_gain_swr(height, diameter, freq):
    nec = sc_quad_helix(height, diameter)
    nec.set_frequency(freq) # TODO: ensure that we don't need to re-generate this!
    nec.radiation_pattern(thetas=Range(90, 90, count=1), phis=Range(180,180,count=1))

    rp = nec.context.get_radiation_pattern(0)
    ipt = nec.get_input_parameters(0)
    z = ipt.get_impedance()[0]

    return ipt.get_frequency(), rp.get_gain()[0], vswr(z, system_impedance)

In [5]:
def create_optimization_target():
    def target(args):
        height, diameter  = args
        if height <= 0 or diameter <= 0:
            print "wrong element dimension"
            return float('inf')

        try:
            result = 0.0

            vswr_score = 0.0
            gains_score = 0.0

            freq, gain, vswr = get_gain_swr(height, diameter, freq=143.05)

            # VSWR should minimal in both bands, gains maximal:
            result = vswr #- gains_score

        except Exception,e:
            print str(e)
            return float('inf')
        print result, height, diameter
        return result
    return target


In [6]:
def simulate_and_get_impedance(nec):
  nec.set_frequency(design_freq_mhz)

  nec.xq_card(0)

  index = 0
  return nec.get_input_parameters(index).get_impedance()[0]  # select only one impedance result (other are the same due to the structure symmetry)

system_impedance = 100 # This makes it a bit harder to optimize, given the 75 Ohm TLs, which is good for this excercise of course...

# (2.4 GHz to 2.5 GHz) and the 5.8 GHz ISM band (5.725 GHz to 5.875 GHz)

design_freq_mhz = 143 # The center of the first range
wavelength = 299792e3/(design_freq_mhz*1000000)

#majorLocator = mpl.ticker.MultipleLocator(10)
#majorFormatter = mpl.ticker.FormatStrFormatter('%d')
#minorLocator = mpl.ticker.MultipleLocator(1)
#minorFormatter = mpl.ticker.FormatStrFormatter('%d')

In [7]:
def draw_frequencie_ranges(ax):
    ax.axvline(x=140, color='red', linewidth=1)
    ax.axvline(x=144, color='red', linewidth=1)

def show_report(height, diameter):
    nec = sc_quad_helix(height, diameter)

    z = simulate_and_get_impedance(nec)

    print "Initial impedance: (%6.1f,%+6.1fI) Ohms" % (z.real, z.imag)
    print "VSWR @ 50 Ohm is %6.6f" % vswr(z, system_impedance)

    nec = sc_quad_helix(height, diameter)
  
    freqs, gains, vswrs = get_gain_swr_range(height, diameter, start=100, stop=200)

    freqs = np.array(freqs) / 1000000 # In MHz
    
    print len(freqs), len(gains), len(vswrs)
  
    ax = plt.subplot(111)
    ax.plot(freqs, gains)
    draw_frequencie_ranges(ax)

    ax.set_title("Gains of a 5-element log-periodic antenna")
    ax.set_xlabel("Frequency (MHz)")
    ax.set_ylabel("Gain")

    #ax.yaxis.set_major_locator(majorLocator)
    #ax.yaxis.set_major_formatter(majorFormatter)

    #ax.yaxis.set_minor_locator(minorLocator)
    #ax.yaxis.set_minor_formatter(minorFormatter)

    #ax.yaxis.grid(b=True, which='minor', color='0.75', linestyle='-')

    plt.show()

    ax = plt.subplot(111)
    ax.plot(freqs, vswrs)
    draw_frequencie_ranges(ax)

    ax.set_yscale("log")
    ax.set_title("VSWR of a QHA antenna @ 100 Ohm impedance")
    ax.set_xlabel("Frequency (MHz)")
    ax.set_ylabel("VSWR")

    #ax.yaxis.set_major_locator(majorLocator)
    #ax.yaxis.set_major_formatter(majorFormatter)
    #ax.yaxis.set_minor_locator(minorLocator)
    #ax.yaxis.set_minor_formatter(minorFormatter)
  
    #ax.yaxis.grid(b=True, which='minor', color='0.75', linestyle='-')
    plt.show()

In [8]:
  initial_height  = wavelength * 0.3
  initial_diameter  = wavelength * 0.2

  print "Wavelength is %0.4fm, initial height and diameter is %0.4fm, %0.4fm" % (wavelength, initial_height, initial_diameter)
  
  print "Unoptimized antenna..."
  show_report(initial_height, initial_diameter)

  print "Optimizing antenna..."
  target = create_optimization_target()

  # Optimize local minimum only with gradient desce
  #optimized_result = scipy.optimize.minimize(target, np.array([initial_height, initial_diameter]), method='Nelder-Mead')

  # Use differential evolution:
  minimizer_kwargs = dict(method='Nelder-Mead')
  bounds = [ (0.2, 0.9), (0.2, 0.9) ]
  #optimized_result = scipy.optimize.differential_evolution(target, bounds, seed=42, disp=True, popsize=20)

  # Basin hopping isn't so good, but could also have been an option:
  optimized_result = scipy.optimize.basinhopping(target, np.array([initial_height, initial_diameter]), minimizer_kwargs=minimizer_kwargs, niter=5, stepsize=0.015, T=2.0, disp=True)

  print "Optimized antenna..."
  optimized_height, optimized_diameter =  optimized_result.x[0], optimized_result.x[1]
  print "Wavelength is %0.4fm, optimized height and diameter is %0.4fm, %0.4fm" % (wavelength, optimized_height, optimized_diameter)
  show_report(optimized_height, optimized_diameter)



Wavelength is 2.0964m, initial height and diameter is 0.6289m, 0.4193m
Unoptimized antenna...
Initial impedance: (   8.9,-904.5I) Ohms
VSWR @ 50 Ohm is 927.962059
101 101 101
Optimizing antenna...
925.992081378 0.628934265734 0.41928951049
869.468013857 0.660380979021 0.41928951049
784.76108962 0.628934265734 0.440253986014
732.448516319 0.660380979021 0.440253986014
645.022594028 0.676104335664 0.450736223776
582.130440485 0.644657622378 0.471700699301
471.388370882 0.636795944056 0.497906293706
373.430252511 0.683966013986 0.508388531469
243.82465699 0.711481888112 0.542455804196
179.004511062 0.672173496503 0.589625874126
110.179725866 0.670208076923 0.659070699301
96.3004934224 0.744894020979 0.70362020979
143.421913008 0.798943059441 0.806477167832
134.423587983 0.70362020979 0.820235104895
108.994030732 0.705585629371 0.75079027972
135.736623586 0.780271573427 0.79533979021
97.668231336 0.697723951049 0.693137972028
98.8357516482 0.737032342657 0.645967902098
94.8493693165 0.7291

/usr/lib/python2.7/dist-packages/scipy/optimize/optimize.py:462: RuntimeWarning: invalid value encountered in subtract
  numpy.max(numpy.abs(fsim[0] - fsim[1:])) <= ftol):


4.13476633645 1.50827065661 0.0987678256795
267.068841783 1.43285712378 0.0987678256795
64.0024951292 1.47056389019 0.097592018231
92.5796135494 1.54597742302 0.0952404033338
15.6477314902 1.4894172734 0.0970041145067
28.7661865051 1.52712403981 0.0958283070581
4.8934662198 1.498843965 0.0967101626446
11.4086254375 1.51769734821 0.0961222589203
2.87175518131 1.5035573108 0.0965631867135
3.88186299752 1.5035573108 0.0918599569192
2.93481788947 1.50473564725 0.0935869241093
3.11875198378 1.50709232015 0.0970408584895
2.72149958771 1.50532481548 0.0944504077043
2.79787114265 1.51003816128 0.0919518168761
2.66861155868 1.50841794866 0.0931046593355
2.76485323693 1.50547210754 0.0934904711545
2.6484686927 1.5061717448 0.0936340023372
2.68597359432 1.50926487799 0.0922882539684
2.61536099982 1.50827986236 0.0928287924023
2.67610067568 1.5060336585 0.0933581354041
2.60904611166 1.50782187612 0.0931680283526
2.85859764696 1.50992999368 0.0923628184178
2.59439665485 1.50711130702 0.093316206357

In [8]:
freqs, gains, vswrs = get_gain_swr_range(0.6, 0.4, start=143, stop=143)

In [9]:
gains

[array([-0.02072662])]

In [14]:
nec = sc_quad_helix(0.6, 0.4)
nec.set_frequency(143.05) # TODO: ensure that we don't need to re-generate this!
nec.radiation_pattern(thetas=Range(90,0, count=180), phis=Range(180,0,count=180))

rp = nec.context.get_radiation_pattern(0)
ipt = nec.get_input_parameters(0)
z = ipt.get_impedance()[0]

In [15]:
  # Gains are in decibels
  gains_db = rp.get_gain()[:,0] # Is an array of theta,phi -> gain. In this case we only have one phi
  thetas = rp.get_theta_angles() * 3.1415 / 180.0
  phis = rp.get_phi_angles() * 3.1415 / 180.0

In [18]:
gains_db

array([-0.0205057 ,  0.05506628,  0.13020735,  0.20485659,  0.27895393,
        0.35244033,  0.42525793,  0.49735025,  0.56866231,  0.63914081,
        0.70873423,  0.77739296,  0.84506943,  0.91171818,  0.97729599,
        1.0417619 ,  1.10507733,  1.16720612,  1.22811457,  1.28777148,
        1.34614819,  1.40321858,  1.45895911,  1.5133488 ,  1.56636925,
        1.6180046 ,  1.66824154,  1.71706926,  1.76447948,  1.81046633,
        1.85502637,  1.89815855,  1.93986412,  1.98014662,  2.01901179,
        2.05646753,  2.09252384,  2.12719273,  2.16048816,  2.19242598,
        2.22302385,  2.25230115,  2.28027888,  2.30697963,  2.33242746,
        2.35664782,  2.37966743,  2.40151426,  2.42221735,  2.44180681,
        2.46031362,  2.47776964,  2.49420742,  2.50966019,  2.52416168,
        2.53774607,  2.55044791,  2.56230195,  2.57334313,  2.58360642,
        2.59312677,  2.60193897,  2.61007762,  2.61757696,  2.62447087,
        2.63079271,  2.63657531,  2.64185079,  2.6466506 ,  2.65